# Some research challenges in plane-wave DFT

## Landscape of DFT codes

- Plane-wave DFT is a wide-spread method
- [Plenty of software](https://en.wikipedia.org/wiki/Vienna_Ab_initio_Simulation_Package) already exists:
  * [ABINIT](https://www.abinit.org) (GPL, Fortran)
  * [CASTEP](http://www.castep.org) (proprietary, Fortran)
  * [VASP](https://www.vasp.at) (proprietary, Fortran)
  * [GPAW](https://wiki.fysik.dtu.dk/gpaw) (GPL, python, C)
  * [Quantum ESPRESSO](https://www.quantum-espresso.org) (GPL, Fortran, C)
  * [KSSOLV](https://crd-legacy.lbl.gov/~chao/KSSOLV) (academic, Matlab)
- Represents hundreds of man-years of coding work!
- Why bother writing a new code?
- Let's look at some current hot topics ...

## Motivating problem: High-throughput screening

- DFT is a key method to simulate material properties:
    - Photovoltaics $\Rightarrow$ solar cells
    - Catalytic properties $\Rightarrow$ energy conversion
    - Material stability
- Nowadays: Discovery of novel materials mostly done *in silico*
- Idea: Build up a **design space** by modifying previous results
- 10000 or more compounds!
- Cut down to the 10-100 most promising
- DFT is a **crucial ingredient**!
- Usually not directly used, but only to train machine-learning model, but that does not help much

**High-throughput requirements:**
- Automatisation (millions of calculations!)
- Reliability (humans need to detect failures / wrong results)
- Efficiency (DFT is not cheap)

- In an ideal world: **black-box DFT workflows**: User choses only accuracy, program decides how to get there

**State of the art:**
- Sizable number of parameters:
    * $k$-point sampling
    * $E_\text{cut}$
    * Multi-layer convergence tolerance (SCF vs. eigensolver)
    * Basically only double precision supported
- Selected by experience / heuristics:
    * **Too conservative** $\Rightarrow$ inferiour performance
    * **Too optimistic** $\Rightarrow$ inferiour reliability
- In practice up to 50% calculations fail:
    * Human supervision
    * Idle time
    * Delay in research results
    
**Multidisciplinary problem:**
- Reliability: rigorous error control, **numerical analysis**, mathematical properties, **physical** model
- Efficiency: error control, numerical analysis, **high-performance computing**
- Automatisation: flexibility of code, **software engineering**
- Research question: **materials science**

## The case for DFTK

<img src="img/dftk_4nodes.png" width=500 align="left" style="padding-right:30px; " />

- Different communites work differently:
    * **Mathematicians:** Toy models and unphysical edge cases
    * **Scientist:** Design new models, not tweak numerics
    * **High-performance specialist:** Exploit what hardware offers
    * **Practicioner:** Reliable, black-box, high-level interface
    
    
    
- Need one code to bundle the efforts!
- $\Rightarrow$ DFTK designed as **platform for** multidisciplinary **collaboration**



- Used by **both** mathematicians and application scientists
- **Performance** within factor 2 to 3 to established codes
- $\Rightarrow$ Not the fastest code, but fast enough to do practical simulation work
- $\Rightarrow$ Still extremely flexible

## The Julia programming language

- https://julialang.org
- Started 10 years ago, 1.0 released in August 2018

- *Walk like Python, talks like Lisp, runs like FORTRAN*
- High-level language, but still hackable
- Just-in-time compiled to byte code before *native* execution
- Key concept from functional languages: **Multiple dispatch**
- Amongst C++ and Fortran one of the 3 languages which have been [scaled to a complete supercomputer](https://juliacomputing.com/blog/2019/04/12/Supercomputing-julia.html)
- Rich ecosystem (Optimisation, PDEs, stochastic processes, GPUs, Machine-Learning, Statistics, Linear Algebra, ...)

- $\Rightarrow$ Greatest advantage is **composability** and **code reuse**:
  * High-level code, automatically hardware-specific kernel
  * E.g. CPU and GPU code look similar to alike
  * Library code and user code decoupled extremely well
  * Simplifies mixing and matching from the ecosystem as needed
  * Key reason for the **rapid development** of DFTK
- Quick one-hour overview: https://github.com/mfherbst/julia-one-hour-intro
- One-day introductory course: https://michael-herbst.com/learn-julia

## Focus problem 1: Reliable SCF algorithms

- Not trivial, focus on a few aspects only.

- One can view the SCF as a fixed-point problem
  $$ F(V(\rho)) = \rho $$
  that is iteratively solved
  $$ \rho_{n+1} = \rho_n + P^{-1} F(V(\rho_n)) $$
- How to choose the preconditioner $P$?
- Depends on **dielectric properties** of a material
- $\Rightarrow$ Need to do SCF differently for metals, insulators, semiconductors, ...

|   Aluminium (a metal)                                     |   Gallium arsenide (a semiconductor)   |   Silica (an insulator)   |
| :-------------------------------------------------------: | :------------------------------------: | :-------: |
| <img src="img/Al_repeat_40.png" width=300 />   | <img src="img/GaAs_repeat_40.png" width=300 /> | <img src="img/SiO2_repeat_39.png" width=300 /> |

- What to do for mixed systems, e.g. Aluminium + Silica:
  <img src="img/alsio2.png" width=500 />
- Turns out standard cases don't do so well:
  <img src="img/AlSiO2H_repeat_20.png" width=400 />

- LDOS is a preconditiong strategy we developed and [published recently](https://michael-herbst.com/publications/2020.09.03_ldos_preconditioning.pdf) using DFTK
- **More details** on this in the [SSD Seminar Talk on DFT](https://blog.rwth-aachen.de/irtg-mip/event/ssd-herbst/) on 12 July 2021.

**The role of DFTK:**
- To obtain mathematical understanding need to treat reduced problems:
  * 1D or 2D
  * Small $E_\text{cut}$ and $k$-grids for fast feedback loop
  
  
- To **design an algorithm**:
  * Mix and match ideas
  * Change order of typical steps (e.g. for backtracking, line searches, ...)
  * No need to be fast, needs to be easy to try
  * $\Rightarrow$ High-level code inside key algorithms
  
  
- To **test an algorithm**:
  * Scale implementation to realistic problem sizes
  * Finer grids, performance optimisation
  * First version never works $\Rightarrow$ Should not take weeks to do this!
  
- **DFTK supports both** (see exercises)


- Subject of ongoing research (**Theses topics** available!)

## Focus problem 2: Error estimation in DFT

- Sources of error in plane-wave DFT:
  * Model error
  * Discretisation error (cutoff and BZ)
  * Algorithm error
  * Arithmetic error
- Natural question to ask: How large?

**State of the art:**
- "Benchmarking": Emprical accuracy idea
- Other methods exist:
    * Uncertainty quantification (aviation, car manifacture)
    * A posteriori error analysis
    * **Not used** in quantum chemistry

**Prospect of error analysis**
- Errors estimated $\Rightarrow$ Parameters chosen to balance errors optimally
    * Parameters automatically chosen $\Rightarrow$ Automatisation
    * Total error controlled $\Rightarrow$ Reliability
    * Optimal error balance $\Rightarrow$ Efficiency
    * **Would solve many key problems** in high-throughput
- Problem: Rigorous error estimation for full DFT is open research problem

**Reduced models:**
- First results [obtained recently](https://michael-herbst.com/publications/2020.04.28_error_nonscf_kohn_sham.pdf)
  on non-SCF models:

<img src="img/si_band_errors.png" width=600 height=600 />

**The role of DFTK:**
- Fully customisable model: Toy problems accessible
- Code agrees with mathematical structure of DFT
- Code independent of model: Results can be expanded to full DFT later!